In [22]:
import jax
import jax.numpy as jnp
import jraph
import sys
import ase
import e3nn_jax as e3nn

sys.path.append('..')
sys.path.append('../analyses')
import analyses.analysis as analysis
from symphony.data import input_pipeline_tf
from symphony import loss, train

In [27]:
workdir = '/data/NFS/potato/songk/spherical-harmonic-net/workdirs/silica-allegro-200k-train-steps-all-dec29'
name = analysis.name_from_workdir(workdir)
_, _, config = analysis.load_model_at_step(
    workdir, "best", run_in_evaluation_mode=True
)

In [7]:
# Load the dataset.
datasets = input_pipeline_tf.get_datasets(0, config)
for step, graphs in enumerate(datasets["test"].as_numpy_iterator()):
    graphs = jax.tree_map(jnp.asarray, graphs)
    for graph in jraph.unbatch(graphs):
        fragment = graph
        break
    break

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 17159314842678948251
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 17159314842678948251
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing 

In [24]:
for step in [str(i) for i in range(30000, 210000, 30000)] + ['200000']:
    model, params, config = analysis.load_model_at_step(
        workdir, step, run_in_evaluation_mode=True
    )
    apply_fn = jax.jit(model.apply)
    eval_metrics = train.evaluate_model(
        model,
        datasets,
        ['test'],
        jax.random.PRNGKey(0),
        config.loss_kwargs,
    )
    print(eval_metrics)

ValueError: Non-hashable static arguments are not supported, as this can lead to unexpected cache-misses. Static argument (index 3) of type <class 'ml_collections.config_dict.config_dict.ConfigDict'> for function evaluate_step is non-hashable.

In [28]:
analysis.load_metrics_from_workdir(workdir)

(add_noise_to_positions: true
 atomic_numbers:
 - 8
 - 14
 compute_padding_dynamically: false
 dataset: silica
 eval_every_steps: 30000
 focus_and_target_species_predictor:
   activation: softplus
   compute_global_embedding: false
   embedder_config:
     activation: softplus
     avg_num_neighbors: 300.0
     max_ell: 5
     mlp_activation: swish
     mlp_n_layers: 2
     model: Allegro
     num_basis_fns: 8
     num_channels: 64
     num_interactions: 4
     r_max: 5
     use_pseudoscalars_and_pseudovectors: false
   latent_size: 128
   num_layers: 3
 frag_pool_size: 1024
 fragment_logic: nn
 freeze_node_embedders: false
 learning_rate: &id001 !!python/object:ml_collections.config_dict.config_dict.FieldReference
   _field_type: &id002 !!python/name:builtins.float ''
   _ops: []
   _required: false
   _value: 0.001
 learning_rate_schedule: constant
 learning_rate_schedule_kwargs:
   decay_steps: 50000
   init_value: *id001
   peak_value: !!python/object:ml_collections.config_dict.con

In [49]:
num_graphs = 1
num_radii = 2
target_positions = jnp.asarray([[1., 1., 1.]])

# RADII = jnp.asarray([1., 2.])

# true_radius_weights = jax.vmap(
#     lambda target_position: jax.vmap(
#         lambda radius: jnp.exp(
#             -((radius - jnp.linalg.norm(target_position)) ** 2)
#             / (2 * 1)
#         )
#     )(RADII)
# )(target_positions)
true_radius_weights = jnp.asarray([[1., 0.]])

position_coeffs = e3nn.IrrepsArray("1o", jnp.asarray([[[1., 1., 1.], [0., 0., 0.]]]))
position_logits = e3nn.to_s2grid(
    position_coeffs,
    res_beta=50,
    res_alpha=39,
    quadrature="gausslegendre",
    normalization="integral",
    p_val=1,
    p_arg=-1,
)
print(position_logits)
position_dist = position_logits.apply(jnp.exp)
integrals = position_dist.integrate().array
print(integrals)
position_dist /= jnp.where(integrals == 0, 1, integrals)
position_logits = position_dist.apply(safe_log)
print(jnp.isnan(position_logits.grid_values).sum())

norms = jnp.linalg.norm(target_positions, axis=-1, keepdims=True)
target_positions_unit_vectors = target_positions / jnp.where(
    norms == 0, 1, norms
)
target_positions_unit_vectors = e3nn.IrrepsArray(
    "1o", target_positions_unit_vectors
)
res_beta, res_alpha, quadrature = (
    position_logits.res_beta,
    position_logits.res_alpha,
    position_logits.quadrature,
)
log_true_angular_dist = e3nn.to_s2grid(
    target_positions_unit_vectors,
    res_beta,
    res_alpha,
    quadrature=quadrature,
    p_val=1,
    p_arg=-1,
)
assert log_true_angular_dist.grid_values.shape == (
    num_graphs,
    res_beta,
    res_alpha,
), log_true_angular_dist.grid_values.shape
#print(log_true_angular_dist, position_logits)

log_true_angular_dist_max = jnp.max(
    log_true_angular_dist.grid_values, axis=(-2, -1), keepdims=True
)
log_true_angular_dist = log_true_angular_dist.apply(lambda x: x - log_true_angular_dist_max)
true_angular_dist = log_true_angular_dist.apply(
    lambda x: jnp.exp(x - log_true_angular_dist_max)
)
true_angular_dist = true_angular_dist / true_angular_dist.integrate()
assert true_angular_dist.grid_values.shape == (num_graphs, res_beta, res_alpha)
print(true_angular_dist.grid_values, position_dist.grid_values)

# Integrate the true angular distribution with the predicted logits.
cross_entropy_at_radius = (
    (true_angular_dist[:, None, :, :] * position_logits)
    .integrate()
    .array.squeeze(axis=-1)
)
assert cross_entropy_at_radius.shape == (num_graphs, num_radii)


radius_normalizing_factors = position_logits.apply(jnp.exp).integrate()
radius_normalizing_factors = radius_normalizing_factors.array.squeeze(axis=-1)
assert radius_normalizing_factors.shape == (
    num_graphs,
    num_radii,
)


def safe_log(x):
    return jnp.log(jnp.where(x == 0, 1.0, x))

lower_bounds = (
    -(true_angular_dist * true_angular_dist.apply(safe_log)).integrate().array.squeeze(axis=-1)
)
lower_bounds += (
    -(true_radius_weights * safe_log(true_radius_weights)).sum(axis=-1)
)

loss_position = jax.vmap(
    lambda qr, fr, Zr, lb: -jnp.sum(qr * fr) + jnp.log(jnp.sum(Zr)) - lb
)(
    true_radius_weights,
    cross_entropy_at_radius,
    radius_normalizing_factors,
    lower_bounds,
)
loss_position

SphericalSignal(shape=(1, 2, 50, 39), res_beta=50, res_alpha=39, quadrature=gausslegendre, p_val=1, p_arg=-1)
[[[[-1.6476383 -1.6354802 -1.6257724 ... -1.6953983 -1.6779903
    -1.6619316]
   [-1.5327659 -1.5049033 -1.4826561 ... -1.6422174 -1.6023237
    -1.5655222]
   [-1.4113327 -1.3677797 -1.3330044 ... -1.5824196 -1.5200607
    -1.462535 ]
   ...
   [ 2.0020342  2.045587   2.0803626 ...  1.8309472  1.8933063
     1.9508318]
   [ 1.910662   1.9385245  1.9607718 ...  1.8012105  1.8411041
     1.8779057]
   [ 1.8125366  1.8246946  1.8344026 ...  1.7647766  1.7821845
     1.7982433]]

  [[ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   ...
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...

AssertionError: 

In [84]:
def kl_on_sphere(true_radial, log_true_angular_coeffs, log_predicted_coeffs):
    log_true_angular_dist = e3nn.to_s2grid(
        log_true_angular_coeffs,
        res_beta,
        res_alpha,
        quadrature=quadrature,
        p_val=1,
        p_arg=-1,
    )
    log_true_angular_dist_max = jnp.max(
        log_true_angular_dist.grid_values, axis=(-2, -1), keepdims=True
    )
    log_true_angular_dist = log_true_angular_dist.apply(lambda x: x - log_true_angular_dist_max)
    true_angular_dist = log_true_angular_dist.apply(
        lambda x: jnp.exp(x - log_true_angular_dist_max)
    )
    true_angular_dist = true_angular_dist / true_angular_dist.integrate()

    true_dist = true_radial * true_angular_dist[None, :, :]
    self_entropy = -(true_dist * true_dist.apply(safe_log)).integrate().array.sum()

    print(e3nn.from_s2grid(true_dist.apply(safe_log), "1o + 2e"), log_predicted_coeffs)
    log_predicted_dist = e3nn.to_s2grid(
        log_predicted_coeffs,
        res_beta,
        res_alpha,
        quadrature=quadrature,
        p_val=1,
        p_arg=-1,
    )
    log_predicted_dist_max = jnp.max(log_predicted_dist.grid_values)
    log_predicted_dist = log_predicted_dist.apply(lambda x: x - log_predicted_dist_max)
    cross_entropy = -(true_dist * log_predicted_dist).integrate().array.sum()
    normalizing_factor = jnp.log(log_predicted_dist.apply(jnp.exp).integrate().array.sum())

    return cross_entropy + normalizing_factor - self_entropy

kl_on_sphere(jnp.asarray([0.9, 0.1]), e3nn.IrrepsArray("1o", jnp.asarray([2., 1., 1.])), e3nn.IrrepsArray("1o", jnp.asarray([[1., 5., 1.], [1., 1., 1.]])))



1x1o+1x2e
[[ 1.9999998e+00  9.9999988e-01  1.0000001e+00 -1.8529313e-08
   0.0000000e+00  1.0430813e-07 -2.9802322e-08  6.1211871e-08]
 [ 1.9999998e+00  1.0000010e+00  1.0000002e+00  9.9738529e-09
   0.0000000e+00 -4.8428774e-07 -8.9406967e-08  9.0113069e-08]] 1x1o
[[1. 5. 1.]
 [1. 1. 1.]]


Array(2.8226643, dtype=float32)

In [15]:
coeffs = e3nn.IrrepsArray("1o", jnp.asarray([1., 1., 1.]))
sig = e3nn.to_s2grid(coeffs, 50, 69, quadrature="soft", p_val=1, p_arg=-1)
go.Surface(sig.plotly_surface(scale_radius_by_amplitude=True))

import plotly.graph_objects as go

go.Figure([go.Surface(sig.plotly_surface(scale_radius_by_amplitude=True))])

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'surfacecolor': array([[-1.7311963, -1.7311963, -1.7311963, ..., -1.7311963, -1.7311963,
                                      -1.7311963],
                                     [-1.6767896, -1.6720676, -1.6678355, ..., -1.6875434, -1.6819625,
                                      -1.6767896],
                                     [-1.5613635, -1.5472164, -1.5345371, ..., -1.5935812, -1.5768611,
                                      -1.5613635],
                                     ...,
                                     [ 1.8873644,  1.9015115,  1.9141908, ...,  1.8551466,  1.8718667,
                                       1.8873644],
                                     [ 1.7856028,  1.7903247,  1.794557 , ...,  1.774849 ,  1.7804298,
                                       1.7856028],
                                     [ 1.7311963,  1.7311963,  1.7311963, ...,  1.7311963,  1.7311963,
                                       1.7311963]], dtype=float32),
              'type': 'surface',
              'x': array([[ 0.        ,  0.        ,  0.        , ..., -0.        , -0.        ,
                            0.        ],
                          [ 0.        ,  0.00477613,  0.0094886 , ..., -0.00960071, -0.00480438,
                            0.        ],
                          [ 0.        ,  0.01324069,  0.02615554, ..., -0.02716189, -0.01349435,
                            0.        ],
                          ...,
                          [ 0.        ,  0.01627265,  0.03262658, ..., -0.03162016, -0.01601893,
                            0.        ],
                          [ 0.        ,  0.00511392,  0.01020954, ..., -0.0100974 , -0.00508564,
                            0.        ],
                          [ 0.        ,  0.        ,  0.        , ..., -0.        , -0.        ,
                            0.        ]], dtype=float32),
              'y': array([[-1.7311963, -1.7311963, -1.7311963, ..., -1.7311963, -1.7311963,
                           -1.7311963],
                          [-1.6759622, -1.6712425, -1.6670125, ..., -1.6867106, -1.6811326,
                           -1.6759622],
                          [-1.5544341, -1.5403498, -1.5277268, ..., -1.5865089, -1.569863 ,
                           -1.5544341],
                          ...,
                          [ 1.8789881,  1.8930726,  1.9056956, ...,  1.8469135,  1.8635592,
                            1.8789881],
                          [ 1.7847216,  1.7894412,  1.7936714, ...,  1.7739732,  1.7795513,
                            1.7847216],
                          [ 1.7311963,  1.7311963,  1.7311963, ...,  1.7311963,  1.7311963,
                            1.7311963]], dtype=float32),
              'z': array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
                           0.        ],
                          [0.05267083, 0.05230489, 0.05152312, ..., 0.05213195, 0.05261442,
                           0.05267083],
                          [0.14693762, 0.14500299, 0.14202468, ..., 0.14748935, 0.14778125,
                           0.14693762],
                          ...,
                          [0.17761709, 0.17820704, 0.17716244, ..., 0.1716978 , 0.17542878,
                           0.17761709],
                          [0.05608883, 0.05600416, 0.05543783, ..., 0.05482901, 0.05569463,
                           0.05608883],
                          [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
                           0.        ]], dtype=float32)}],
    'layout': {'template': '...'}
})